In [8]:
!pip install transformers
!pip install torch
!pip install pandas
!pip install scikit-learn
!pip install tensorflow  # If you want to use TensorFlow instead of PyTorch

  Using cached transformers-4.42.4-py3-none-any.whl (9.3 MB)


In [7]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer

# Load the dataset
data = pd.read_excel('./full_courtdata.xlsx')

# Concatenate relevant text columns for input
data['text_input'] = data['मुद्दाको किसिम'] + ' ' + data['वादी'] + ' ' + data['प्रतिबादी'] + ' ' + data['विवरण']

# Handle missing values
data['text_input'].fillna('', inplace=True)

# Label encoding
label_encoder = LabelEncoder()
data['labels'] = label_encoder.fit_transform(data['आदेश /फैसलाको किसिम'])

# Select features and labels
features = data['text_input']
labels = data['labels']


ModuleNotFoundError: No module named 'transformers'

In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import DataLoader, Dataset
import torch

class CourtDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')
model = AutoModelForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=len(label_encoder.classes_))

# Tokenize the text
encodings = tokenizer(list(features), truncation=True, padding=True, max_length=128)

# Create a dataset
dataset = CourtDataset(encodings, labels)


ModuleNotFoundError: No module named 'transformers'

In [16]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

# Load pre-trained model
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=len(label_mapping))

# Set up training arguments
training_args = TrainingArguments(
    output_dir='./results',          # Output directory
    num_train_epochs=3,              # Number of training epochs
    per_device_train_batch_size=8,   # Batch size for training
    per_device_eval_batch_size=8,    # Batch size for evaluation
    warmup_steps=500,                # Number of warmup steps
    weight_decay=0.01,               # Strength of weight decay
    logging_dir='./logs',            # Directory for storing logs
    logging_steps=10,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

# Train the model
trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [ ]:
# %pip uninstall keras
# %pip install tf-keras
# %pip install tensorflow



In [14]:
# %pip install transformers[torch]
# %pip install accelerate -U

In [15]:
%pip show accelerate transformers torch


Name: accelerate
Version: 0.32.1
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: zach.mueller@huggingface.co
License: Apache
Location: /home/oceanic/Documents/fuse/project/.env/lib/python3.10/site-packages
Requires: huggingface-hub, numpy, packaging, psutil, pyyaml, safetensors, torch
Required-by: 
---
Name: transformers
Version: 4.42.4
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /home/oceanic/Documents/fuse/project/.env/lib/python3.10/site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: 
---
Name: torch
Version: 2.3.1
Summa

In [ ]:
# Evaluate the model
eval_results = trainer.evaluate()

print(f"Evaluation results: {eval_results}")


In [ ]:
model.save_pretrained('./nepali_bert_model')
tokenizer.save_pretrained('./nepali_bert_tokenizer')


In [ ]:
def predict(text, model, tokenizer):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)
    return predictions.item()

# Example prediction
new_text = "Your new input text in Nepali"
predicted_label = predict(new_text, model, tokenizer)
label = [key for key, value in label_mapping.items() if value == predicted_label][0]

print(f"Predicted label: {label}")
